Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, we'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

In [34]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [41]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.75)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))


assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"
    

There are 57340 sentences in the corpus.
There are 43005 sentences in the training set.
There are 14335 sentences in the testing set.


#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [53]:
key = 'b100-28144'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-28144
words:
	('and', 'August', '15', ',', 'November', '15', ',', 'February', '17', ',', 'and', 'May', '15', ',', '(', 'Cranston', ')', '.')
tags:
	('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [43]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 870604 samples of 49082 unique words in the training set.
There are 290588 samples of 28205 unique words in the testing set.
There are 6975 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [7]:
# accessing words with Dataset.X and tags with Dataset.Y
for i in range(3):
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')

Sentence 3: ('Such', 'an', 'instrument', 'is', 'expected', 'to', 'be', 'especially', 'useful', 'if', 'it', 'could', 'be', 'used', 'to', 'measure', 'the', 'elasticity', 'of', 'heavy', 'pastes', 'such', 'as', 'printing', 'inks', ',', 'paints', ',', 'adhesives', ',', 'molten', 'plastics', ',', 'and', 'bread', 'dough', ',', 'for',

#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [46]:

# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 80: break



Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')
	 ('and', 'CONJ')
	 ('now', 'ADV')
	 ('he', 'PRON')
	 ('made', 'VERB')
	 ('use', 'NOUN')
	 ('of', 'ADP')
	 ('the', 'DET')
	 ('advice', 'NOUN')
	 ('.', '.')
	 ('But', 'CONJ')
	 ('there', 'PRT')
	 ('seemed', 'VERB')
	 ('to', 'PRT')
	 ('be', 'VERB')
	 ('some', 'DET')
	 ('difference', 'NOUN')
	 ('of', 'ADP')
	 ('opinion', 'NOUN')
	 ('as', 'ADP')
	 ('to', 'ADP')
	 ('how', 'ADV')
	 ('far', 'ADV')
	 ('the', 'DET')
	 ('board', 'NOUN')
	 ('should', 'VERB')
	 ('go', 'VERB')
	 (',', '.')
	 ('and', 'CONJ')
	 ('whose', 'DET')
	 ('advice', 'NOUN')
	 ('it', 'PRON')
	 ('should', 'VERB')
	 ('follow', 'VERB')
	 ('.', '.')
	 ('Such', 'PRT')
	 ('an', 'DET')
	 ('instrument', 'NOUN')
	 ('is', 'VERB')
	 ('expected', 'VERB')
	 ('to', 'PRT')
	 ('be', 'VERB')
	 ('especially', 'ADV')
	 ('useful', 'ADJ')
	 ('if', 'ADP')
	 ('it', 'PRON')
	 ('could', 'VERB')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [47]:
from collections import Counter

def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    
    
    results = {}
    
    for x1, sentence in enumerate(sequences_A):
        for x2, s_A in enumerate(sentence):
            
            t = sequences_B[x1][x2]
            
            if s_A in results and t in results[s_A]:
                results[s_A][t] += 1
            elif s_A in results:
                    results[s_A][t] = 1
            else:
                results[s_A] = {t:1}

    return results
    
emission_counts = pair_counts(data.training_set.Y, data.training_set.X)

#print(len(emission_counts))
assert len(emission_counts) == 12, \
       "There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')


In [48]:
emission_counts["NOUN"]

{'artists': 33,
 'representations': 7,
 'figures': 66,
 'B.C.': 13,
 'reality': 62,
 'Plate': 2,
 'months': 133,
 'defendant': 5,
 'world': 514,
 'individuals': 52,
 'grave': 11,
 'fathers': 8,
 'mothers': 20,
 'graybeards': 1,
 'adolescents': 5,
 'babies': 7,
 'death': 201,
 'meaning': 89,
 'gentlemen': 17,
 'point': 269,
 'time': 1198,
 'Alleghenies': 1,
 'Poconos': 1,
 'Pennsylvania': 33,
 'Renovo': 1,
 'Foliage': 4,
 'Festival': 12,
 'Oct.': 19,
 'doubt': 72,
 'Tim': 17,
 'Mrs.': 412,
 'Buck': 11,
 "cowboy's": 1,
 'name': 201,
 'cow': 18,
 'milk': 35,
 'pitcher': 13,
 'lure': 4,
 'marriage': 68,
 'trap': 11,
 'domesticity': 1,
 'cats': 10,
 'energy': 68,
 'thermocouple': 2,
 'temperature': 97,
 'gas': 65,
 'ideals': 13,
 'country': 253,
 'Protestantism': 9,
 'stock': 96,
 'Matsuo': 23,
 'rifle': 49,
 'Rousseau': 14,
 'Dave': 27,
 'Trager': 1,
 'showman': 3,
 'boss': 12,
 "Chicago's": 5,
 'basketball': 7,
 'Packers': 1,
 'club': 52,
 'use': 269,
 'season': 70,
 'ticket': 8,
 'holder

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [50]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word

from collections import namedtuple
import operator

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

word_counts = pair_counts(data.training_set.X, data.training_set.Y)

mfc_table = {}

for word in data.training_set.vocab:
    tag_val = max(word_counts[word].keys(), key=(lambda k: word_counts[word][k]))        
    mfc_table[word] = tag_val

mfc_model = MFCTagger(mfc_table)


In [51]:
mfc_table

{'spraying': 'VERB',
 'shimmering': 'VERB',
 'Arrangement': 'NOUN',
 'event': 'NOUN',
 'Spectacular': 'ADJ',
 'dined': 'VERB',
 'under-developed': 'ADJ',
 'roadbed': 'NOUN',
 'relationships': 'NOUN',
 'dusty': 'ADJ',
 'yellowish': 'ADJ',
 'thoroughfare': 'NOUN',
 'Status-roles': 'NOUN',
 'oblong': 'ADJ',
 'Westerners': 'NOUN',
 'Australian': 'NOUN',
 'Greville': 'NOUN',
 'Wansee': 'NOUN',
 'canker': 'NOUN',
 'Curiously': 'ADV',
 'Weidman': 'NOUN',
 'generalities': 'NOUN',
 "Susan's": 'NOUN',
 '357': 'NUM',
 'Rag': 'NOUN',
 'Tunisia': 'NOUN',
 'roaring': 'VERB',
 'Samples': 'NOUN',
 'self': 'NOUN',
 'rudely': 'ADV',
 'hard-to-please': 'ADJ',
 'farmlands': 'NOUN',
 'crowd': 'NOUN',
 'part': 'NOUN',
 'posse': 'NOUN',
 'Katie': 'NOUN',
 'clamps': 'NOUN',
 'Hanover-Pebble': 'NOUN',
 'buoyancy': 'NOUN',
 "Colt's": 'NOUN',
 'god-like': 'ADJ',
 'discharged': 'VERB',
 'occurs': 'VERB',
 'launched': 'VERB',
 'unload': 'VERB',
 'synthetics': 'NOUN',
 'Eligio': 'NOUN',
 'duffers': 'NOUN',
 'Zanzib

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [52]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [55]:
for key in data.testing_set.keys[:5]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-23173

Predicted labels:
-----------------
['NOUN', '.', 'ADV', '.', 'VERB', 'ADP', 'ADP', 'NOUN', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'CONJ', 'ADV', 'ADJ', 'NOUN', '.', 'NOUN', '.', 'PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CONJ', 'PRON', 'PRON', 'VERB', 'VERB', 'PRT', 'VERB', 'ADV', 'ADP', 'NOUN', 'PRT', 'NOUN', 'ADP', 'NOUN', '.', 'ADV', 'PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', '<MISSING>', '<MISSING>', 'NOUN', 'PRT', 'ADP', 'DET', '<MISSING>', 'NOUN', 'ADP', 'NOUN', '.']

Actual labels:
--------------
('NOUN', '.', 'ADV', '.', 'VERB', 'ADP', 'DET', 'NOUN', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'CONJ', 'ADV', 'ADJ', 'NOUN', '.', 'NOUN', '.', 'PRON', 'VERB', 'ADV', 'ADV', 'DET', 'ADJ', 'NOUN', 'CONJ', 'PRON', 'PRON', 'VERB', 'VERB', 'PRT', 'VERB', 'ADV', 'ADP', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', '.', 'ADV', 'PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'ADV', 'ADP', 'DET', 'NOUN'

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [56]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [57]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))


training accuracy mfc_model: 95.73%
testing accuracy mfc_model: 92.93%


Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).


### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [58]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
        
    # problem is that you have many sequences
    # also you have to divide by N
    unigram_counts = {}

    for i, sentence in enumerate(sequences):
        for j, y in enumerate(sentence):
            if y in unigram_counts:
                unigram_counts[y] += 1
            else:
                unigram_counts[y] = 1
    
    return unigram_counts

# TODO: call unigram_counts with a list of tag sequences from the training set

tag_unigrams = unigram_counts(data.training_set.Y)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [25]:

def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """

    
    bigram_counts = {}
    
    for i, sentence in enumerate(sequences):
        for y in range(len(sentence) - 1):
            if (sentence[y], sentence[y+1]) in bigram_counts:
                bigram_counts[(sentence[y], sentence[y+1])] += 1
            else:
                bigram_counts[(sentence[y], sentence[y+1])] = 1
    
    
    
    return bigram_counts

# TODO: call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(data.training_set.Y)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [26]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    
    starting_counts = {}
    
    for i, sentence in enumerate(sequences):
        if sentence[0] in starting_counts:
            starting_counts[sentence[0]] += 1
        else:
            starting_counts[sentence[0]] = 1


    return starting_counts

# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.training_set.Y)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [28]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    ending_counts = {}
    
    for i, sentence in enumerate(sequences):
        indexVal = len(sentence) -1
        if sentence[indexVal] in ending_counts:
            ending_counts[sentence[indexVal]] += 1
        else:
            ending_counts[sentence[indexVal]] = 1
    
    return ending_counts

# TODO: Calculate the count of each tag ending a sequence
tag_ends = ending_counts(data.training_set.Y)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [29]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

states = {}

for tag in emission_counts:
    prob_dct = {}
    total = tag_unigrams[tag]
    prob_dist = {key: value / total for key, value in emission_counts[tag].items()}
    
    tag_distribution = DiscreteDistribution(prob_dist)
    temp_state = State(tag_distribution, name=tag)
    states[tag] = temp_state
    
    basic_model.add_state(states[tag])
    
for bigram in tag_bigrams.keys():
    
    len_data_y = len(data.training_set.Y)
    b0 = bigram[0]
    b1 = bigram[1]
    
    prob =  tag_starts[b0] / len_data_y
    basic_model.add_transition(basic_model.start, states[b0], prob)
    
    prob = tag_bigrams[bigram] / tag_unigrams[b0] 
    basic_model.add_transition(states[b0], states[b1], prob)
    

    prob = tag_ends[b0] / len_data_y
    basic_model.add_transition(states[b0], basic_model.end, prob)

basic_model.bake()



assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [32]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct!</div>')

training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 95.96%


### Example Decoding Sequences with the HMM Tagger

In [33]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 